In [1]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_census_income
import tensorflow.keras.backend as K
import argparse
from scalelayer import  ScaleLayer

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred


def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

models_map = {
    'a': "models/gated_models/adult_a_gated_4_0.3_0.2_p-0.3_p0.15.h5",
    'r': "models/gated_models/adult_r_gated_4_0.3_0.2_p-0.95_p0.8.h5",
    'g': "models/gated_models/adult_g_gated_4_0.3_0.2_p-0.6_p0.1.h5",
    'a&r': "models/gated_models/adult_a&r_gated_4_0.3_0.2_p-0.35_p0.25.h5",
    'a&g': "models/gated_models/adult_a&g_gated_4_0.3_0.2_p-0.3_p0.25.h5",
    'r&g': "models/gated_models/adult_r&g_gated_4_0.3_0.2_p-0.9_p0.8.h5",
}

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'layer5'), ('layer5', 'layer6')]

from preprocessing import pre_census_income
from tensorflow.keras.models import Model

for layer_index in range(5, 6):
    print("*"*10, "layer", layer_index+1)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]
        
        model_path = models_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        print(model.get_layer('layer6').get_weights())
#         model.summary()

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_census_income.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_census_income.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_census_income.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()
        
        for m, out in enumerate(inter_output_ori):
            print("index:",m, out )

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

********** layer 6
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[array([[ 2.9423175 ],
       [-1.499268  ],
       [-0.73832774],
       [-0.76415855],
       [ 0.704021  ],
       [-4.6148725 ],
       [-0.95385844],
       [-0.68753624],
       [-1.078599  ],
       [ 1.7334886 ]], dtype=float32), array([-0.18941183], dtype=float32)]
inter_output_ori [[1.7130086e-01]
 [1.7318130e-04]
 [8.5446358e-02]
 ...
 [4.9315524e-01]
 [2.6781598e-01]
 [6.3598223e-02]]
attr: a 0.0
[array([[ 1.0153329 ],
       [-0.9641135 ],
       [-0.31642342],
       [-0.5854113 ],
       [ 0.05338898],
       [ 0.6885379 ],
       [-0.5771605 ],
       [-0.10637736],
       [-0.07460939],
       [ 0.39883998]], dtype=float32), array([-0.7370

KeyboardInterrupt: 

In [3]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_bank_marketing
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_bank_marketing.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            }

model_path = "models/gated_models/bank_a_gated_4_0.3_0.2_p-0.15_p0.3.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('dense_132', 'scale_layer_5'), ('dense_133', 'scale_layer_6'), ('dense_134', 'scale_layer_7'), ('dense_135', 'scale_layer_8'), ('dense_136', 'layer5')]

from preprocessing import pre_bank_marketing
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_bank_marketing.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_bank_marketing.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_bank_marketing.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                510       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)                15  

In [8]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_census_income
import tensorflow.keras.backend as K
import argparse
from scalelayer import  ScaleLayer

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred


def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

model_map = {
    'a': "models/diff_adult_a_gated_4_diff.h5",
    'r': "models/diff_adult_r_gated_4_diff.h5",
    'g': "models/diff_adult_g_gated_4_diff.h5",
    'a&r': "models/diff_adult_a&r_gated_4_diff.h5",
    'a&g': "models/diff_adult_a&g_gated_4_diff.h5",
    'r&g': "models/diff_adult_r&g_gated_4_diff.h5",
}

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'layer5')]

from preprocessing import pre_census_income
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]
        
        model_path = model_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        model.summary()

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_census_income.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_census_income.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_census_income.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))
                                 

layer 0
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)            

attr: a&g 0.096
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)    

attr: a&r 0.028
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)    

attr: g 0.001
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r 0.001
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: a 0.003
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r&g 0.053


In [8]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_bank_marketing
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_bank_marketing.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            }

model_path = "models/gated_models_diff/diff_bank_a_gated_4_diff.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('dense_132', 'scale_layer_5'), ('dense_133', 'scale_layer_6'), ('dense_134', 'scale_layer_7'), ('dense_135', 'scale_layer_8'), ('dense_136', 'layer5')]

from preprocessing import pre_bank_marketing
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_bank_marketing.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_bank_marketing.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_bank_marketing.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                510       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)                15  

In [4]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_lsac
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(50, activation="relu", input_shape=pre_lsac.X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(int(constraint[i][0]), int(constraint[i][1]+1)))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 
            'r': [10],
            'g': [9],
            'g&r': [10, 9],
            }

models_map = {
            'r': 'models/gated_models/lsac_r_gated_4_0.3_0.2_p-0.9_p0.05.h5',
            'g': 'models/gated_models/lsac_g_gated_4_0.3_0.2_p-0.85_p0.2.h5',
            'g&r': 'models/gated_models/lsac_g&r_gated_4_0.3_0.2_p-0.55_p0.45.h5',
}



# similar_X = [dis_data, new_data]

layer_map = [('layer5', 'scale_layer_5'), ('layer5', 'scale_layer_6'), ('layer5', 'scale_layer_7'), ('layer5', 'scale_layer_8'), ('layer5', 'layer5')]

from preprocessing import pre_lsac
from tensorflow.keras.models import Model

for layer_index in range(0, 5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)
        model_path = models_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        model.summary()

        dis_data = pre_lsac.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_lsac.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_lsac.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

layer 0
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)            

attr: g 0.089
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r 0.072
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: g&r 0.073


In [9]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_lsac
import tensorflow.keras.backend as K
import argparse
from scalelayer import  ScaleLayer

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(50, activation="relu", input_shape=pre_lsac.X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(int(constraint[i][0]), int(constraint[i][1]+1)))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 
            'r': [10],
            'g': [9],
            'g&r': [10, 9],
            }

models_map = {
            'r': 'models/gated_models_diff/diff_lsac_r_gated_4_diff.h5',
            'g': 'models/gated_models_diff/diff_lsac_g_gated_4_diff.h5',
            'g&r': 'models/gated_models_diff/diff_lsac_g&r_gated_4_diff.h5',
}



# similar_X = [dis_data, new_data]

layer_map = [('layer5', 'scale_layer_5'), ('layer5', 'scale_layer_6'), ('layer5', 'scale_layer_7'), ('layer5', 'scale_layer_8'), ('layer5', 'layer5')]

from preprocessing import pre_lsac
from tensorflow.keras.models import Model

for layer_index in range(0, 5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)
        model_path = models_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = tf.keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        model.summary()

        dis_data = pre_lsac.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_lsac.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_lsac.X_train)

        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()
        
        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum()

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

layer 0
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)            

attr: g 0.121
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r 0.063
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: g&r 0.024
